## Imports

In [81]:
import pandas as pd
import altair as alt
import tabula

## Data read-in

In [82]:
# Read in data and filter to desired DF
dfs = tabula.read_pdf('2023-news-release-july-1st-preliminary-taxable-values.pdf',pages=2)
df = dfs[0]

In [83]:
# Rename columns
df = df.rename(columns={'Unnamed: 0':'TAXING AUTHORITY',
                                '2022':'2022 PRELIMINARY TAXABLE VALUE',
                                '2023':'2023 TAXABLE VALUE BEFORE NEW CONSTRUCTION',
                               'Unnamed: 1':'PERCENT CHANGE',
                               'Unnamed: 2':'NEW CONSTRUCTION',
                               '2023.1':'2023 PRELIMINARY TAXABLE VALUE',
                               'Unnamed: 3':'PERCENT CHANGE FROM 2022',
                               'Unnamed: 4':'NET VALUE CHANGE'})

In [84]:
# Remove bad rows
df = df.drop(index=[0,1,2,40])

In [85]:
# Reset index
df = df.reset_index()

In [86]:
# Drop old index
df = df.drop(columns='index')

In [87]:
df

,TAXING AUTHORITY,2022 PRELIMINARY TAXABLE VALUE,2023 TAXABLE VALUE BEFORE NEW CONSTRUCTION,PERCENT CHANGE,NEW CONSTRUCTION,2023 PRELIMINARY TAXABLE VALUE,PERCENT CHANGE FROM 2022,NET VALUE CHANGE
0,01 MIAMI,"73,749,481,211","83,042,917,479",12.6%,"1,408,850,944","84,451,768,423",14.5%,"10,702,287,212"
1,0101 MIAMI ( DDA ),"24,602,533,803","27,528,563,602",11.9%,"171,237,637","27,699,801,239",12.6%,"3,097,267,436"
2,02 MIAMI BEACH,"46,544,694,070","51,346,874,323",10.3%,"213,897,907","51,560,772,230",10.8%,"5,016,078,160"
3,0201 MB NORMANDY SHORES,"305,472,912","389,416,157",27.5%,"3,369,531","392,785,688",28.6%,"87,312,776"
4,03 CORAL GABLES,"20,101,009,587","22,188,650,684",10.4%,"454,317,791","22,642,968,475",12.6%,"2,541,958,888"
5,04 HIALEAH,"15,369,871,099","17,249,692,407",12.2%,"300,274,188","17,549,966,595",14.2%,"2,180,095,496"
6,05 MIAMI SPRINGS,"1,449,806,131","1,598,589,815",10.3%,"6,968,534","1,605,558,349",10.7%,"155,752,218"
7,06 NORTH MIAMI,"4,605,444,600","5,199,015,711",12.9%,"61,074,883","5,260,090,594",14.2%,"654,645,994"
8,07 NORTH MIAMI BEACH,"4,113,425,952","4,751,311,110",15.5%,"-767,739","4,750,543,371",15.5%,"637,117,419"
9,08 OPA-LOCKA,"1,551,388,190","1,681,633,415",8.4%,"36,607,979","1,718,241,394",10.8%,"166,853,204"
